In [1]:
# import the necessary libraries
import numpy as np
import sys, math, cv2

In [2]:
# Load the image
img = cv2.imread("develop-012.jpg")
#cv2.imshow('Input Image', img)
#cv2.waitKey(0)

error: OpenCV(4.5.4) /Users/runner/work/opencv-python/opencv-python/opencv/modules/highgui/src/window.cpp:1006: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


Convert the image to a different colorspace (HSV)

In [ ]:
# convert to hsv space
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#cv2.imshow('Input Image in HSV', hsv)
#cv2.waitKey(0)

The next step is to remove the blue background from the image while in HSV colorspace by defining the upper and lower bounds of the color blue and using opencv inRange function to convert all the vlaues within that range to black

In [ ]:
# remove blue from the image
upper_blue = np.array([135,255,255])  #-- upper range --
lower_blue = np.array([85,50,0])  #-- lower range --
mask = cv2.inRange(hsv, lower_blue, upper_blue)
res = cv2.bitwise_not(hsv, hsv, mask= mask)  #-- Contains pixels having the gray color--

img[mask!=0] =[0,0,0]
cv2.imshow('Image without Blue',img)
cv2.waitKey(0)

This is a method to crop the black background by reading the image, converting it to grayscale by thresholding and finding the contours and cropping the image by using a bounding rectangle. It is not very effective, however, since the map isn't a 90 degree rectangle so there are black areas still within the bounding rectangle.

In [ ]:
# crop the image
#gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#_,thresh = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)

#contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#cnt = contours[0]
#x,y,w,h = cv2.boundingRect(cnt)

#crop = img[y:y+h,x:x+w]
#cv2.imwrite("develop-001-cropped.jpg", crop)

#cv2.imshow('Result',crop)
#cv2.waitKey(0)

This next step find the contours of the dark part of the image to segment the map from the background.
read image and convert to grayscale, then apply binary thresholding to get an idea of where the edges are located. Using a threshold value that makes most of the map white is ideal because the border will be detected easily.
the next step is to find and draw the contours using the drawContours() method to overlay the contours on the original image

In [ ]:
# contour and crop
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# apply binary thresholding
# Any pixel with a value greater than 150 will be set 
# to a value of 255 (white)
ret, img_thresh = cv2.threshold(img_gray, 10, 255, cv2.THRESH_BINARY)
# visualize the binary image
cv2.imshow('Binary image', img_thresh)
cv2.waitKey(0)
#cv2.imwrite('thresh1.jpg', img_thresh)
#cv2.destroyAllWindows()

The CHAIN_APPROX_SIMPLE  algorithm compresses horizontal, vertical, and diagonal segments along the contour and leaves only their end points. This means that any of the points along the straight paths will be dismissed, and we will be left with only the end points. For example, consider a contour, along a rectangle. All the contour points, except the four corner points will be dismissed. This method is faster than the CHAIN_APPROX_NONE because the algorithm does not store all the points, uses less memory, and therefore, takes less time to execute.

In [ ]:
# detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
contours, hierarchy = cv2.findContours(image=img_thresh, 
                                       mode=cv2.RETR_TREE, 
                                       method=cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    epsilon = 0.1 * cv2.arcLength(c, True)
    approx_rectangle = cv2.approxPolyDP(c, epsilon, True)
    if len(approx_rectangle) == 4:
        break
        
# draw contours on the original image
image_copy = img.copy()
cv2.drawContours(image=image_copy, contours=c, 
                 contourIdx=0, color=(0, 255, 0), 
                 thickness=2, lineType=cv2.LINE_AA)
               
# see the results
cv2.imshow('Image with Contours', image_copy)
cv2.waitKey(0)
#cv2.imwrite('contours_none_image1.jpg', image_copy)
#cv2.destroyAllWindows()

Use perspective transform to reshape the map by providing the points on the image from which to gather information and wrap the original image

In [ ]:
#epsilon = 0.1 * cv2.arcLength(contours[0], True)
#approx_rectangle = cv2.approxPolyDP(contours[0], epsilon, True)
#print(approx_rectangle)

In [ ]:
a = approx_rectangle[0][0]
b = approx_rectangle[1][0]
c = approx_rectangle[2][0]
d = approx_rectangle[3][0]

In [ ]:
img_pts = cv2.drawContours(img, approx_rectangle, -1, (0, 255, 0), 3)
cv2.imshow("Image with Edge Points", img_pts)
cv2.waitKey(0)

In [ ]:
wid_ad = np.sqrt(((a[1] - d[1])**2) + ((a[0] - d[0])**2))
wid_cb = np.sqrt(((c[1] - b[1])**2) + ((c[0] - b[0])**2))
len_dc = np.sqrt(((d[1] - c[1])**2) + ((d[0] - c[0])**2))
len_ab = np.sqrt(((a[1] - b[1])**2) + ((a[0] - b[0])**2))
print(wid_ad)
print(wid_cb)
print(len_dc)
print(len_ab)

max_width = max(int(wid_ad), int(wid_cb))
max_length = max(int(len_dc), int(len_ab))
print(max_width, max_length)

In [ ]:
wid_ad

In [ ]:
# Locate points of the documents
# or object which you want to transform
# pts1 is the original coordinates of the image
# pts2 is the coordinates used to transform the image
pts1 = np.float32([b, c, d, a])

pts2 = np.float32([[0,0],
                   [0, max_width],
                   [max_length, max_width],
                   [max_length, 0]])

# Apply Perspective Transform Algorithm
matrix = cv2.getPerspectiveTransform(pts1, pts2)
img_cropped = cv2.warpPerspective(img_pts, matrix, ( max_length,max_width))
cv2.imshow("Image Cropped & Transformed", img_cropped)
cv2.waitKey(0)

Next step is to find the contour of the green arrow

In [ ]:
# finding the green contour runs inside this loop
run = 1
i = 0
while run:
    # convert again to hsv space
    hsv = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2HSV)
    img_pts = 0
    # remove green from the image
    upper_green = np.array([75, 255, 255]) #-- upper range --
    lower_green = np.array([45, 100, 100])  #-- lower range --
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    # detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
    contours, hierarchy = cv2.findContours(image=mask_green, 
                                           mode=cv2.RETR_TREE, 
                                           method=cv2.CHAIN_APPROX_NONE)
    #################
    # draw the contours around the arrow to see the points 
    img_pts = cv2.drawContours(img_cropped, contours, -1, (255, 0, 0), 3)
    cv2.imshow('Image Rotated Upright', img_pts)
    cv2.waitKey(0)
    # write a for loop to find the biggest contour
    for c in contours:
        epsilon = 0.1 * cv2.arcLength(c, True)
        approx_polygon = cv2.approxPolyDP(c, epsilon, True)
        if len(approx_polygon) > 3 and cv2.contourArea(c) < 1000:
            break
    arrow_region = approx_polygon[0][0]   
    print(arrow_region)
    if arrow_region[0] > 700 and arrow_region[1] < 500:
        run = 0   
    else:
        img_cropped = cv2.rotate(img_cropped, cv2.ROTATE_180)
        #i = i + 1
        #if i == 2:
        #run = 0
    print(cv2.contourArea(c))   
        # add an exception handler that stops the code 
        # after rotating 3 times other wise it might rotate 
        # infinitely with some images
# see the results
cv2.imshow('Image Rotated Upright', img_cropped)
cv2.waitKey(0)

In [ ]:

Lower_Red_1 = np.array([0,100,30]    ,dtype= "uint8") 
Upper_Red_1 = np.array([30,255,255], dtype="uint8")

Lower_Red_2 = np.array([145,100,30]    ,dtype= "uint8") 
Upper_Red_2 = np.array([225,255,255], dtype="uint8")


hsv = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2HSV)

mask = cv2.inRange(hsv ,  Lower_Red_1, Upper_Red_1) + cv2.inRange(hsv,  Lower_Red_2, Upper_Red_2)

cv2.imshow('Map+Contour',mask)
cv2.waitKey(0)

contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


print(contours)


for i,c in  enumerate(contours):    
    peri   = cv2.arcLength(c, True)
    approx_triangle = cv2.approxPolyDP(c, 0.04 * peri, True)
    
    if len(approx_triangle)== 3:
        break
     
    
#print(approx)


Contour_Map = cv2.drawContours(img_cropped, c, 1,(0,255,0), 1)

cv2.imshow('Map+Contour',Contour_Map)
cv2.waitKey(0)



To remove red

In [ ]:
# convert to hsv space
hsv = cv2.cvtColor(img_cropped, cv2.COLOR_BGR2HSV)
# remove red from the image
# red has two regions on the color spectrum
lower_red1 =np.array([0,100,30]    ,dtype= "uint8") 
upper_red1 = np.array([30,255,255], dtype="uint8")
lower_red2 = np.array([145,100,30]    ,dtype= "uint8") 
upper_red2 = np.array([225,255,255], dtype="uint8")
# generate the lower and upper mask of red
mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
# merge the masks
#mask_red = cv2.bitwise_or(mask_red1, mask_red2)
mask_red = mask_red1+mask_red2
    
# detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
contours, hierarchy = cv2.findContours(image=mask_red, 
                                       mode=cv2.RETR_TREE, 
                                       method=cv2.CHAIN_APPROX_SIMPLE)
cv2.imshow('Image with Red Mask', mask_red)
cv2.waitKey()

In [ ]:
#mask_red1+mask_red2
#cv2.imshow('Image with Red Mask', mask_red)
#cv2.waitKey()

In [ ]:
# write a for loop to find the contour
#for i, c in enumerate(contours):
    #epsilon = cv2.arcLength(c, True)
   # approx_triangle = cv2.approxPolyDP(c, 0.4*epsilon, True)
    #if len(approx_triangle) == 3:
        #break

# draw contours on the original image
#image_copy = img_cropped.copy()
#cv2.drawContours(image=image_copy, contours=c, 
                 #contourIdx=c, color=(0, 255, 0), 
                 #thickness=2, lineType=cv2.LINE_AA)
#gh =cv2.drawContours(img_cropped, c, 1,(0,255,0), 5)              
# see the results
#cv2.imshow('Image with Contours', gh)
#cv2.waitKey(0)

In [ ]:
contours

In [ ]:
# The red pointer is an isosceles triangle, by finding the length
# of the sides, we can determine from the lengths with roughly equal 
# sides which one is the pointer and use that to find the position
a = approx_triangle[0][0]
b = approx_triangle[1][0]
c = approx_triangle[2][0]

ab = np.sqrt(((a[1] - b[1])**2) + ((a[0] - b[0])**2))
bc = np.sqrt(((c[1] - b[1])**2) + ((c[0] - b[0])**2))
ca = np.sqrt(((c[1] - a[1])**2) + ((c[0] - a[0])**2))

print(ab)
print(bc)
print(ca)

In [ ]:
# since the two sides of the triangle are roughly the same size and both geeater than 130,
# the smallest side is the opposite of the pointer
if ab > 100 and bc > 100:
    point = b
elif bc > 100 and ca > 100:
    point = c
elif ca > 100 and ab > 100:
    point = a
print(point)

In [ ]:
if ab < 130:
    midpoint = (a+b)/2
elif bc < 130:
    midpoint = (b+c)/2
elif ca < 130:
    midpoint = (a+c)/2 
print(midpoint)

In [ ]:
# to find the position of the pointer:
xposs = point[0]
yposs = point[1]

xpos = xposs/max_length
ypos = 1 - (yposs/max_width)
print(f"POSITION {xpos:.3f} {ypos:.3f}")

To calculate bearing, we will use trigonometric functions and the coordinates from the red pointer

In [ ]:
x = midpoint
y = point
hdg = ((np.arctan2((y[1]-x[1]), (y[0]-x[0]))*180/np.pi))+90
print(f"POSITION {xpos:.3f} {ypos:.3f}")
print(f"BEARING {hdg:.1f}")

In [ ]:
#math.atan2((y[1]-x[1]), (y[0]-x[0]))

In [ ]:
#math.degrees(math.atan2((y[1]-x[1]), (y[0]-x[0])))+90

In [ ]:
# https://stackoverflow.com/questions/67962200/python-opencv-perspective-transformation-problems

In [ ]:
# https://www.geeksforgeeks.org/perspective-transformation-python-opencv/

In [ ]:
# https://www.geeksforgeeks.org/python-opencv-cv2-rotate-method/

In [ ]:
# https://www.thoughtco.com/understanding-the-distance-formula-2312242